<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/RAG_lessons/blob/main/Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile requirements.txt
llama-index
llama-index-llms-huggingface
llama-index-embeddings-huggingface
Unstructured[md]
einops
accelerate
sentence-transformers
bitsandbytes
llama-index-postprocessor-colbert-rerank

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

In [3]:
!pip install pypdf

In [4]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.postprocessor.colbert_rerank import ColbertRerank

reader = SimpleDirectoryReader(input_files=["/content/Size_of_Countries.pdf"])
data = reader.load_data()

In [6]:
from llama_index.core import PromptTemplate
system_prompt = "Please answer the questions in a truthful and reliable way."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base", trust_remote_code=True)

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=hf_token,
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=8192,
    max_new_tokens=512,
    model_kwargs={
        "token": hf_token,
        "torch_dtype": torch.bfloat16,
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": False,
        "temperature": 0.05,
        "top_p": 0.3,

    },
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_kwargs={"token": hf_token},
    stopping_ids=stopping_ids,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
from llama_index.core import Settings


Settings.embed_model = embed_model

Settings.chunk_size = 1024
# Llama-3-8B-Instruct model
Settings.llm = llm

In [11]:
index = VectorStoreIndex.from_documents(data)

colbert_reranker = ColbertRerank(
    top_n=3,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

In [12]:
query_engine = index.as_query_engine(similarity_top_k=3, streaming=True, node_postprocessors=[colbert_reranker],)



In [17]:
response = query_engine.query(r'''
According to this article, what are the benefits of size in terms of population for countries?
''')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [18]:
print(response)

According to the article, the main benefits of size in terms of population for countries are:

1. Economies of scale in the production of public goods. The per capita cost of many public goods is lower in larger countries, where more taxpayers pay for them.
2. A larger country (both in terms of population and national product) is less subject to foreign aggression. Thus, safety is a public good that increases with country size.
3. Larger countries can better internalize cross-region externalities by centralizing the provision of those public goods that involve strong externalities.


Chúng ta có thể kiểm tra kết quả của quá trình reranking, bằng cách in ID và nội dung của từng node một cùng với retrieval score. Ở trường hợp này thì Llama-3 chỉ tìm được 2 node cho nội dung chúng ta cần tìm.

In [15]:
for node in response.source_nodes:
    print(node.id_)
    print(node.node.get_content())
    print("reranking score: ", node.score)
    print("retrieval score: ", node.node.metadata["retrieval_score"])
    print("=====================================")

f41b6d89-4126-41d6-8003-1a14cfc96af2
Ch. 23: Trade, Growth and the Size of Countries 1505
completely clear.12Whatever the source of the border effect, however, the correlation
between the “political size” of a country and its market size does not totally disappear
even in the absence of policy-induced trade barriers. Still, one would expect that the
correlation between size and economic success is mediated by the trade regime. In a
regime of free trade, small countries can prosper, while in a world of trade barriers,
being large is much more important for economic prosperity, measured for instance byincomeper capita.
2.1.2. Thecosts ofsize
If size only had beneﬁts, then the world should be organized as a single political entity.
This is not the case. Why? As countries become larger and larger, administrative andcongestion costs may overcome the beneﬁts of size pointed out above. However, these
typesofcostsbecomebindingonlyforverylargecountriesandtheyarenotlikelytobe
relevant determinan